In [1]:
#########################
#         import        #
#########################
import datetime
import pandas
from pandas_datareader import data
import matplotlib.pyplot as plt
import mplfinance as mpf
from tkinter import *
from tkinter import ttk
from tkcalendar import Calendar, DateEntry

In [2]:
#########################
#     Global params     #
#########################
ExStockID = 0

In [12]:
#########################
#       Stock Show      #
#########################

def Stock_Show(df):
    root = Toplevel()
    root.title("matplotlib 埋め込みテスト")

    '''
    plt.figure(figsize=(6,4))
    df['Close'].plot(label='[Company Name]')
    df['Close'].rolling(5).mean().plot(label='5', linestyle='dashed')
    df['Close'].rolling(25).mean().plot(label='25', linestyle='dashed')
    df['Close'].rolling(75).mean().plot(label='75', linestyle='dashed')
    plt.legend()
    plt.grid()
    dfGraph = pandas.DataFrame({'[Company Name]':df['Close']})
    dfGraph.plot(title='[Company Name]',grid=True)
    plt.savefig("img/test.png")
    '''

    mpf.plot(df ,type='candle', figsize=(6.5,5), mav=(5, 25, 75), #mav：移動平均線の表示
volume=True, style='yahoo',
savefig = dict(fname='img/test.png', dpi=100))

    fig = PhotoImage(file="img/test.png")

    canvas = Canvas(root, bg="white", height=500, width=700)
    canvas.pack()
    canvas.create_image(0, 0, image=fig, anchor=NW)

    root.mainloop()

In [4]:
#########################
#       Get Stock       #
#########################
# SoftBank Stock data from Stooq.com 
# reference https://seanmemo.com/234/

def Get_Stock(Ccode, date_data):
    # 企業コード設定
    Ccode = Ccode + '.JP'

    # サイト設定
    Site = 'stooq'

    # day
    tempdate = date_data.split('/')
    StartD = '20' + tempdate[2] + '-' + tempdate[0] + '-' + tempdate[1]
    date_data = datetime.date.today()
    EndD = str(date_data.year) + '-' + str(date_data.month) + '-' + str(date_data.day)

    df = data.DataReader(Ccode, Site, StartD, EndD)
    print('Getting Stock finished')

    Stock_Show(df)

In [5]:
#########################
#    Stock Exoroler     #
#########################
def StockIdSet(id):
    global ExStockID
    ExStockID = id.split(',')[0]

def Stock_Exproler(keyword):
    root = Toplevel()
    root.title("検索テスト")

    with open("data/companylist.csv") as f:
        lines = f.readlines()

    stocks = [line.strip() for line in lines]

    stock_XXX = [line for line in stocks if keyword in line]
    var = StringVar(value=stock_XXX)

    listbox = Listbox(root, listvariable=var)
    listbox.grid(row=0, column=0)

    StockIndButton = ttk.Frame(root, padding=(0, 5))
    StockIndButton.grid(row=2, column=0, sticky=N)
    buttonInd= ttk.Button(
        StockIndButton,
        text='セット',
        command=lambda: [StockIdSet(listbox.get(listbox.curselection())), root.quit(), root.destroy()]
        )
    buttonInd.grid(row=0, column=1)

    root.mainloop()
    #print(stock_XXX)

In [6]:
#########################
#      Stock Index      #
#########################

def Stock_Index(Userid):
    # set up
    root = Toplevel()
    root.title("test_index")
    root.resizable(False, False)
    frameEntry = ttk.Frame(root)
    frameEntry.grid()

    # Corp Explore label
    labelCorpName = ttk.Label(
        frameEntry,
        text='CorpName'
        )
    labelCorpName.grid(row=0, column=0)

    # Corp ID label
    labelCorpid = ttk.Label(
        frameEntry,
        text='CorpID'
        )
    labelCorpid.grid(row=1, column=0)

    # Day label
    labelDayindex = ttk.Label(
        frameEntry,
        text='Start Day'
        )
    labelDayindex.grid(row=2, column=0)

    # Corp Name Entry
    Corpname = StringVar()
    Corpname_entry = ttk.Entry(
        frameEntry,
        textvariable = Corpname,
        width=20
        )
    Corpname_entry.grid(row=0, column=1)

    # Corp ID Entry
    Corpid = StringVar()
    Corpid_entry = Entry(
        frameEntry,
        textvariable = Corpid,
        width=20
        )
    Corpid_entry.grid(row=1, column=1)

    
    # Day Entry
    calender_date = Calendar(frameEntry, date_patternstr="y-mm-dd")
    calender_date.grid(row=2, column=1)

    # Button Setup
    StockExploreButton = ttk.Frame(frameEntry, padding=(0, 5))
    StockExploreButton.grid(row=0, column=2, sticky=W)
    buttonExplore= ttk.Button(
        StockExploreButton,
        text='検索',
        command=lambda: [Stock_Exproler(Corpname.get()), Corpid_entry.delete(0, END), Corpid_entry.insert(0, ExStockID)]
        )
    buttonExplore.pack(side=LEFT)
    
    StockDrawButton = ttk.Frame(frameEntry, padding=(0, 5))
    StockDrawButton.grid(row=3, column=1, sticky=W)
    buttonDraw= ttk.Button(
        StockDrawButton,
        text='DRAW',
        command=lambda: Get_Stock(Corpid.get(), calender_date.get_date())
        )
    buttonDraw.pack(side=LEFT)

    labelUser = ttk.Label(
        frameEntry,
        text='User: ' + Userid
        )
    labelUser.grid(row=4, column=2)
    
    root.mainloop()

In [7]:
#########################
#      User_Action      #
#########################

def User_Action(Userid):
    root = Toplevel()
    root.title("テスト")
    root.resizable(False, False)
    frameEntry = ttk.Frame(root)
    frameEntry.grid()

    labelUser = ttk.Label(
        frameEntry,
        text='User: ' + Userid
        )
    labelUser.grid(row=0, column=0)

    StockExploreButton = ttk.Frame(frameEntry, padding=(0, 5))
    StockExploreButton.grid(row=1, column=0, sticky=N)
    buttonExplore= ttk.Button(
        StockExploreButton,
        text='株式検索',
        command=lambda: Stock_Index(Userid)
        )
    buttonExplore.pack(side=LEFT)

    StockTradeButton = ttk.Frame(frameEntry, padding=(0, 5))
    StockTradeButton.grid(row=2, column=0, sticky=N)
    buttonTrade= ttk.Button(
        StockTradeButton,
        text='株式売買(未実装)'
        )
    buttonTrade.pack(side=LEFT)

    StockLogoutButton = ttk.Frame(frameEntry, padding=(0, 5))
    StockLogoutButton.grid(row=3, column=0, sticky=N)
    buttonLogout= ttk.Button(
        StockLogoutButton,
        text='ログアウト',
        command=lambda: root.destroy()
        )
    buttonLogout.pack(side=LEFT)

    root.mainloop()
    

In [8]:
#########################
#      login_window     #
#########################

def Login_Window():
    # set up
    root = Tk()
    root.title("test_login(未実装)")
    root.resizable(False, False)
    frameEntry = ttk.Frame(root)
    frameEntry.grid()

    # label
    labelUserid = ttk.Label(
        frameEntry,
        text='UserID'
        )
    labelUserid.grid(row=0, column=0)

    labelPsw = ttk.Label(
        frameEntry,
        text='Password'
        )
    labelPsw.grid(row=1, column=0)

    # ID Entry
    Userid = StringVar()
    Userid_entry = ttk.Entry(
        frameEntry,
        textvariable = Userid,
        width=20
    )
    Userid_entry.grid(row=0, column=1)

    Psw = StringVar()
    Psw_entry = ttk.Entry(
        frameEntry,
        textvariable = Psw,
        width=20,
        show='*'
    )
    Psw_entry.grid(row=1, column=1)

    frameEntryButton = ttk.Frame(frameEntry, padding=(0, 5))
    frameEntryButton.grid(row=2, column=1, sticky=W)

    buttonLogin = ttk.Button(
        frameEntryButton,
        text='OK',
        command=lambda: User_Action(Userid.get())
        )
    buttonLogin.pack(anchor='center')

    buttonCancel = ttk.Button(
        frameEntryButton,
        text='Cancel',
        command=quit
        )
    buttonCancel.pack(anchor='center')

    root.mainloop()

In [9]:
#########################
#          main         #
#########################

#Login_Window()
#User_Action()
Stock_Index('123')

#Get_Stock('9766', '2023-6-1')
#Stock_Exproler('A')

Getting Stock finished


In [10]:
'''
# ttkテスト
import tkinter as tk
root = tk.Tk()
root.config(width=500,
            height=200,
            bg='#000000')
root.mainloop()
https://kuroro.blog/python/zTeIS7KyJGQv87n1kue5/
'''

"\n# ttkテスト\nimport tkinter as tk\nroot = tk.Tk()\nroot.config(width=500,\n            height=200,\n            bg='#000000')\nroot.mainloop()\nhttps://kuroro.blog/python/zTeIS7KyJGQv87n1kue5/\n"